In [ ]:
import pandas as pd;import numpy as np
import matplotlib.pyplot as plt;import seaborn as sns
import matplotlib.dates as mdates
from matplotlib.lines import Line2D

In [ ]:
df=pd.read_excel('//kaggle/input/assign/Assignment_Dataset.xlsx')
df.head()
df.isna().sum()#checking for null values

In [ ]:
# Calculating the budget line dynamically
budget_values = [73.9]
years_since_start = (df['Date'] - df['Date'].min()).dt.days // 365
budget_line = np.zeros_like(years_since_start, dtype=float)
for i in range(max(years_since_start)+1):
    if i == 0:
        budget_line[years_since_start == i] = budget_values[0]
    else:
        budget_line[years_since_start == i] = budget_values[0] - 0.6 * i
        budget_values.append(budget_values[0] - 0.6 * i)
df['PR_MA'] = df['PR'].rolling(window=30).mean()
# Defining the conditions and corresponding colors and labels
conditions = [(df['GHI'] < 2), (df['GHI'] < 4), (df['GHI'] < 6), (df['GHI'] >= 6)]
colors = ['navy', 'lightblue', 'orange', 'brown']
labels = ['< 2', '2~4', '4~6', '>6']
df['Condition'] = np.select(conditions, labels)

# Scatterplot
plt.figure(figsize=(10, 6))
sns.scatterplot(x=df['Date'], y=df['PR'], hue=df['Condition'], marker='D', palette=colors, legend=False)
plt.plot(df['Date'], df['PR_MA'], color='red')
plt.xlabel('')
plt.ylabel('Performance Ratio[%]')
plt.title('Performance Ratio Evolution \nFrom 2019-07-01 to 2022-03-24')
a = np.sum(df['PR_MA'] > budget_line)
b = df['PR_MA'].count()
# Creating label dynamically
label = 'Target Budget Yield Performance Ratio['
for i in range(max(years_since_start) + 1):
    label += f'{i+1}Y-{budget_values[i]:.2f}%,'
label = label.rstrip(',') + ']'

plt.plot(df['Date'], budget_line, color='green', linestyle='-')

# Creating a custom legend for the lines
line_legend_elements = [ Line2D([0], [0], color='green', lw=2, label=label)
                        ,Line2D([0], [0], color='red', lw=2, label='30-d moving average of PR')]
line_legend = plt.legend(handles=line_legend_elements, loc='center')
for text, color in zip(line_legend.get_texts(), ['green', 'red']):
    text.set_color(color);text.set_weight='bold'
# Adding the text
plt.text(0.46, 0.42, f'Points above Target Budget PR = {a}/{b}={(a/b)*100:.2f}%', horizontalalignment='center', verticalalignment='center', transform=plt.gca().transAxes,fontsize=10,weight='bold')

# Creating a custom legend for the scatterplot
scatter_legend_elements = [Line2D([0], [0], marker='D', color='w', markerfacecolor=c, markersize=6,label =l) for c,l in zip(colors,labels)]
legend_elements = [Line2D([0], [0], marker=None, color='w', label='Daily Irradiation [kWh/m2]')] + scatter_legend_elements
scatter_legend = plt.legend(handles=legend_elements, loc='upper center', bbox_to_anchor=(0.40, 0.95), ncol=len(colors)+1)
plt.gca().add_artist(line_legend)

# Creating label dynamically
label = 'Target Budget Yield Performance Ratio['
for i in range(max(years_since_start) + 1):
    label += f'{i+1}Y-{budget_values[i]:.2f}%,'
label = label.rstrip(',') + ']'

plt.plot(df['Date'], budget_line, color='green', linestyle='-')

# Creating a custom legend for the lines
line_legend_elements = [ Line2D([0], [0], color='green', lw=2, label=label)
                        ,Line2D([0], [0], color='red', lw=2, label='30-d moving average of PR')]
line_legend = plt.legend(handles=line_legend_elements, loc='center')
for text, color in zip(line_legend.get_texts(), ['green', 'red']):
    text.set_color(color);text.set_weight='bold'
# Adding the text
plt.text(0.46, 0.42, f'Points above Target Budget PR = {a}/{b}={(a/b)*100:.2f}%', horizontalalignment='center', verticalalignment='center', transform=plt.gca().transAxes,fontsize=10,weight='bold')

# Creating a custom legend for the scatterplot
scatter_legend_elements = [Line2D([0], [0], marker='D', color='w', markerfacecolor=c, markersize=6,label =l) for c,l in zip(colors,labels)]
legend_elements = [Line2D([0], [0], marker=None, color='w', label='Daily Irradiation [kWh/m2]')] + scatter_legend_elements
scatter_legend = plt.legend(handles=legend_elements, loc='upper center', bbox_to_anchor=(0.40, 0.95), ncol=len(colors)+1)
plt.gca().add_artist(line_legend)

# Formating the x-axis
date_fmt = '%b/%y'
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter(date_fmt))
major_locator = mdates.MonthLocator(interval=3)
plt.gca().xaxis.set_major_locator(major_locator)
plt.gca().spines['bottom'].set_position(('data', -5))

# Formating y-axis
plt.yticks(np.arange(0, 101, 10))
plt.xlim(df['Date'].iloc[0], df['Date'].iloc[-1])
plt.ylim(-5, 100)
plt.grid(True, color='lightgrey', linewidth=0.5)

# Calculations
last_7_days_avg = df['PR'].tail(7).mean()
last_30_days_avg = df['PR'].tail(30).mean()
last_60_days_avg = df['PR'].tail(60).mean()
last_90_days_avg = df['PR'].tail(90).mean()
last_365_days_avg = df['PR'].tail(365).mean()

plt.text(0.94, 0.25, f"Average PR last 7-d: {last_7_days_avg:.2f} %", transform=plt.gca().transAxes, ha='right', va='bottom')
plt.text(0.95, 0.20, f"Average PR last 30-d: {last_30_days_avg:.2f} %", transform=plt.gca().transAxes, ha='right', va='bottom')
plt.text(0.95, 0.15, f"Average PR last 60-d: {last_60_days_avg:.2f} %", transform=plt.gca().transAxes, ha='right', va='bottom')
plt.text(0.95, 0.10, f"Average PR last 90-d: {last_90_days_avg:.2f} %", transform=plt.gca().transAxes, ha='right', va='bottom')
plt.text(0.96, 0.05, f"Average PR last 365-d: {last_365_days_avg:.2f} %", transform=plt.gca().transAxes, ha='right', va='bottom')
plt.savefig("assign.png")
plt.show()